**Initial the package and googledrive**

In [0]:
#prepare google drive
from google.colab import drive
drive.mount('/content/gdrive')
!mkdir -p /content/gdrive/My\ Drive/Colab\ Notebooks/files/documents
%cd /content/gdrive/My\ Drive/Colab\ Notebooks
!pwd

#HTML -> PDF
!apt-get update
!apt-get install wkhtmltopdf
!apt-get install xvfb

#install additional fonts
!apt-get install fonts-thai-tlwg  #Thai fonts
!apt-get install fonts-takao-mincho  #Japanese fonts


Download and convert Wikipedia page into PDF file.

In [0]:
from bs4 import BeautifulSoup
import urllib3
import csv
import os

#========== Save Wikipedia page to PDF files ===========

http = urllib3.PoolManager()
urllib3.disable_warnings()

# #### get Keyword files ####
WikiKeywordURL = "https://raw.githubusercontent.com/Eit-wc/Document-Clustering-/master/src/files/Wiki_Keywords.csv"
WikiKeywordsFileName = "./files/Wiki_Keywords.csv"
os.system(f"wget '{WikiKeywordURL}' -O '{WikiKeywordsFileName}'")

def savePDF(url,filename):
  print(f'save {url} -> {filename}')
  os.system(f"xvfb-run wkhtmltopdf '{url}' ./files/documents/{filename}")

def getWikiPage(Keyword,Languages,label):
  global http

  # Access english page
  url = f"https://en.wikipedia.org/wiki/{Keyword}"
  print(f'Request: {url}')
  enPage = http.request('GET', url)

  if(enPage.status == 200):

    # Download english page to PDF
    savePDF(url, f'{label}_{Keyword}_en.pdf')

    # Find other languages
    soupEnPage = BeautifulSoup(enPage.data, 'html.parser')
    # loop other language
    for language in Languages:
      trageTag = soupEnPage.find_all('a',{'class' : 'interlanguage-link-target', 'lang':language})
      if(len(trageTag) >0):

        # Find other language url
        languageURL = trageTag[0].get('href')
        print(f'traget URL: {languageURL}')

        # Download page to PDF
        savePDF(languageURL, f'{label}_{Keyword}_{language}.pdf')
      else:
        print(f'The wikipage have no {language} language')
  else:
    print(f"Cannot access {url}")
  
  
#read CSV
with open(WikiKeywordsFileName) as WikiKeywordsFile:
    WikiKeywords = csv.reader(WikiKeywordsFile, delimiter=',')
    next(WikiKeywords, None)  # skip the headers
    i=1
    #Loop each Keyword
    for Keyword in WikiKeywords:
      if(len(Keyword)>0):
        print(f'{i}: {Keyword}')
        i+=1
        getWikiPage(Keyword[0],['th','ja'],Keyword[1])
